In [1]:
import requests
import pandas as pd
import os
import numpy as np
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

unique_committees = ['labor', 'rules', 'health', 'energy', 'budget', 'banking', 'science', 'finance', 'housing', 'commerce', 'pensions', 'forestry', 'nutrition', 'education', 'agriculture', 'publicworks', 'environment', 'urbanaffairs', 'armedservices', 'appropriations', 'transportation', 'foreignrelations', 'homelandsecurity', 'naturalresources', 'governmentalaffairs', 'jointcongressionalcommitteeoninaugural']
#^note, I copied this list after running the script before having this list, it can be re-created in this way with modifications to the code

url = 'https://www.senate.gov/committees/committee_assignments.htm'

import requests
from bs4 import BeautifulSoup
import PyPDF2

reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
from pathlib import Path





urls = []
for link in soup.find_all('a'):
    if str(link.get('href'))[-3:]=="pdf":
        l = str(link.get('href'))
        if l[0]=='/':
            l="https://www.senate.gov/"+l
        response = requests.get(l)
        filename = Path('committee_pdfs/'+l.split('/')[-1]).write_bytes(response.content)

\nVerDate Aug 31 2005 03:07 Mar 15, 2006 Jkt 026433 PO 00000 Frm 00031 Fmt 6608 Sfmt 6608 E:\\HR\\OC\\26433.XXX 26433ycherry on PROD1PC64 with HEARING

\nVerDate Aug 31 2005 03:07 Mar 15, 2006 Jkt 026433 PO 00000 Frm 00030 Fmt 6608 Sfmt 6608 E:\\HR\\OC\\26433.XXX 26433ycherry on PROD1PC64 with HEARING

In [2]:
def dfFromPDF(pdf):
    pdfFileObj = open(pdf, 'rb')
    pdf = PyPDF2.PdfReader(pdfFileObj)

    text=""
    for page in pdf.pages:
        text=text+'\n'+page.extract_text().split('\nVerDate')[0]
    year = (pdf.pages[0].extract_text().split('VerDate ')[1].split(' ')[2])

    # closing the pdf file object
    pdfFileObj.close()
    


    text=text.split('COMMITTEE ASSIGNMENTS OF SENATORS')[1]
    text=text.split('LIST OF SENATORS BY STATES ')[0]
    for i in range(10):
        text=text.replace(str(i), '')
    # text.replace('Mrs. ', ' WOMAN')

    

    # print(repr(text))
    text = text.replace('.','')
    
    
    text = text.replace(' (chairman)','')
    text=text.lower()
    for s in state_names:
        text=text.replace('of '+s.lower(),'')
    text = text.replace('  ',' ')
    text=text.replace('chairman','')
    text=text.replace('(','')
    text=text.replace(')','')
    entries = text.split(' \n')

    # entries = str(entries).replace('\n', '').replace("'",'').replace('[','').replace('[','').split(', ')

    member_list = []
    for e in range(len(entries)):
        member_dict = {}
        if(entries[e][0:2]=='mr' or entries[e][0:3]=='mrs'):
            

            #gender
            gender = 'M'
            if (entries[e][0:3]=='mrs'):
                gender='W'
            member_dict['gender'] = gender
            name = entries[e][3:].lower()

            #year
            member_dict['year']=int(year)

            #name
            if(name[0]==' '):
                name=name[1:]
            name=name.split(' ')[0]
            # name=name.replace('\n','')
            name=name.capitalize()
            member_dict['name'] = name

            #committees
            committees = entries[e][4:].split(' ',1)[1].replace(', and ', ', ').replace(' and ', ', ').split(', ')
            topop = []
            for c in range(len(committees)):
                committees[c]=committees[c].replace(' ','')
                if(committees[c]=='armedservice'):
                    committees[c]='armedservices'
                if(committees[c] not in str(unique_committees)):
                    topop.append(c)
            for c in topop:
                committees.pop(c)
            member_dict['committees'] = committees

            member_list.append(member_dict)

    df = pd.DataFrame(data=member_list)
    return df

print(dfFromPDF('committee_pdfs/GPO-CPUB-109spub20.pdf'))

   gender  year       name                           committees
0       M  2005      Akaka                      [armedservices]
1       M  2005  Alexander                             [budget]
2       M  2005     Allard                     [appropriations]
3       M  2005      Allen  [commerce, science, transportation]
4       M  2005     Baucus   [agriculture, nutrition, forestry]
..    ...   ...        ...                                  ...
87      M  2005      Thune                      [armedservices]
88      M  2005     Vitter  [commerce, science, transportation]
89      M  2005  Voinovich           [environment, publicworks]
90      M  2005     Warner                      [armedservices]
91      M  2005      Wyden                             [budget]

[92 rows x 4 columns]


In [3]:


directory = 'committee_pdfs'
dfs = []
for filename in os.listdir(directory):
    if str(filename)[0:3]=='GPO':
        dfs.append(dfFromPDF('committee_pdfs/'+str(filename)))

df = pd.concat(dfs)
df.to_csv("committees.csv")
df.head()


,gender,year,name,committees
0,M,2005,Akaka,[armedservices]
1,M,2005,Alexander,[budget]
2,M,2005,Allard,[appropriations]
3,M,2005,Allen,"[commerce, science, transportation]"
4,M,2005,Baucus,"[agriculture, nutrition, forestry]"


In [4]:
df = pd.read_csv("committees.csv", usecols=['gender', 'year', 'name', 'committees'])
df

,gender,year,name,committees
0,M,2005,Akaka,['armedservices']
1,M,2005,Alexander,['budget']
2,M,2005,Allard,['appropriations']
3,M,2005,Allen,"['commerce', 'science', 'transportation']"
4,M,2005,Baucus,"['agriculture', 'nutrition', 'forestry']"
...,...,...,...,...
1590,M,2014,Warnock,"['agriculture', 'nutrition', 'forestry']"
1591,M,2014,Whitehouse,['budget']
1592,M,2014,Wicker,['armedservices']
1593,M,2014,Wyden,['budget']


In [27]:
def getCommitteesByName(name, min_year=2005, max_year=2023):
    list_of_lists = (df.loc[df[(df['name'].str.contains(name.lower().capitalize())) & (df['year']>=min_year) & (df['year']<=max_year)].index, 'committees'])
    print(((list_of_lists.explode())))
    result = {x for l in list_of_lists for x in l}
    return result

print(getCommitteesByName('Booker'))

unique_committees_series = (pd.Series(sorted(list({x for l in df['committees'] for x in l}),key=len)))
unique_committees_series=unique_committees_series[unique_committees_series.str.contains('mr')==False]
unique_committees=[]
for c in unique_committees_series:
    add=True
    if len(unique_committees)>0:
        
        for l in unique_committees:
            if l in c:
                # add=False
                print()
    if(add==True):
        unique_committees.append(c)

str(unique_committees)


917     ['commerce', 'science', 'transportation']
1003    ['commerce', 'science', 'transportation']
1176    ['commerce', 'science', 'transportation']
1264               ['environment', 'publicworks']
1349    ['commerce', 'science', 'transportation']
1436               ['environment', 'publicworks']
1517     ['agriculture', 'nutrition', 'forestry']
Name: committees, dtype: object
{'e', 'p', "'", ']', 'r', 'b', 'n', 'c', 'm', ' ', 's', ',', 'k', 'i', 'f', 'o', '[', 'g', 'y', 'v', 't', 'a', 'u', 'l', 'w'}


'[\'e\', \'p\', \'j\', "\'", \']\', \'r\', \'b\', \'d\', \'n\', \'c\', \'h\', \'m\', \' \', \'s\', \',\', \'k\', \'i\', \'f\', \'o\', \'[\', \'g\', \'y\', \'v\', \'t\', \'a\', \'u\', \'l\', \'w\']'